# Eat Safe, Love

## Notebook Set Up

In [8]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

[]


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
pprint(count)
# Display the first document in the results using pprint
pprint(establishments.find_one())

0
None


In [9]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(establishments.find(query))
# Display the number of rows in the DataFrame
print(len(df))
# Display the first 10 rows of the DataFrame
df.head(10)

0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': '4'}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query2)
# Display the first document in the results using pprint
pprint(establishments.find_one(query2))

None


In [11]:
# Convert the result to a Pandas DataFrame
df2 = pd.DataFrame(establishments.find(query2))
# Display the number of rows in the DataFrame
print(len(df2))
# Display the first 10 rows of the DataFrame
df2.head(10)

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [13]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = .08384

query3 = {'RatingValue' : '5', 'geacode.latitude' : {'$lte' : (latitude + degree_search), '$gte' : (latitude - degree_search)}, 
          'geocode.longitude' : {'$lte' : (longitude + degree_search), '$gte' : (longitude - degree_search)}}
sort =  [{'scores.Hygiene', 1}]

# Print the results
results = list(establishments.find(query3).sort(sort).limit(5))
pprint(results)

TypeError: first item in each key pair must be an instance of str

In [ ]:
# Convert result to Pandas DataFrame
df3 = pd.DataFrame(establishments.find(query3))
df3.head(10)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene': 0}}

group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}

sort_query = {'$sort': {'count': -1}}

pipeline = [match_query, group_query, sort_query]

results2 = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print (len(results2))
# Print the first 10 results
pprint(results2[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
agg_df = pd.json_normalize(results2)
# Display the number of rows in the DataFrame
print(len(agg_df))
# Display the first 10 rows of the DataFrame
agg_df.head(10)